## Motion Planning through KOMO ##

In [1]:
import robotic as ry
import numpy as np
import time
import os
os.environ['DISPLAY'] = ':0'

## Part a) Adding a constraint to simulation

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

First, solve the task without collision constraint

In [3]:
C.setJointState(qHome)
#(config, something with frames, robot speed,?,?)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.0625, evals: 15, done: 1, feasible: 1, sos: 1.91679, f: 0, ineq: 0, eq: 0.00143212 }
[[-3.72622209e-02 -4.52749606e-01 -3.79159291e-02 -2.00577309e+00
  -8.94814888e-03  2.01389406e+00 -4.99968271e-01]
 [-7.45657906e-02 -4.05447715e-01 -7.58679711e-02 -2.01154783e+00
  -1.79003189e-02  2.02780033e+00 -4.99936725e-01]
 [-1.11952006e-01 -3.58042992e-01 -1.13892299e-01 -2.01732601e+00
  -2.68606437e-02  2.04173097e+00 -4.99905544e-01]
 [-1.49462065e-01 -3.10484418e-01 -1.52025208e-01 -2.02310965e+00
  -3.58334814e-02  2.05569791e+00 -4.99874906e-01]
 [-1.87137029e-01 -2.62721413e-01 -1.90303184e-01 -2.02890110e+00
  -4.48235245e-02  2.06971287e+00 -4.99844986e-01]
 [-2.25017799e-01 -2.14703928e-01 -2.28762981e-01 -2.03470314e+00
  -5.38359067e-02  2.08378724e+00 -4.99815953e-01]
 [-2.63145110e-01 -1.66382491e-01 -2.67441706e-01 -2.04051904e+00
  -6.28763059e-02  2.09793214e+00 -4.99787972e-01]
 [-3.01559542e-01 -1.17708213e-01 -3.06376916e-01 -2.04635258e+00
  -7.19510399e-02  2

In [4]:
del komo
del C

Then, we add collision constraint with "accumulatedCollisions" feature

In [5]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g')) #robot arm
#                                          size                 x  y   z
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [6]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1]) #difference is here
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.5, evals: 89, done: 1, feasible: 1, sos: 3.14134, f: 0, ineq: 0, eq: 0.0666667 }
[[-3.66513291e-02 -4.50936007e-01 -3.82874753e-02 -2.00291224e+00
  -9.52815622e-03  2.00952288e+00 -4.98362830e-01]
 [-7.33327290e-02 -4.01816056e-01 -7.66166992e-02 -2.00581677e+00
  -1.90768967e-02  2.01903859e+00 -4.96721003e-01]
 [-1.10074253e-01 -3.52584112e-01 -1.15029484e-01 -2.00870583e+00
  -2.86669271e-02  2.02853992e+00 -4.95069837e-01]
 [-1.46905920e-01 -3.03183997e-01 -1.53567767e-01 -2.01157161e+00
  -3.83191962e-02  2.03801954e+00 -4.93404593e-01]
 [-1.83857695e-01 -2.53559307e-01 -1.92273676e-01 -2.01440620e+00
  -4.80550180e-02  2.04746999e+00 -4.91720449e-01]
 [-2.20959477e-01 -2.03653339e-01 -2.31189589e-01 -2.01720154e+00
  -5.78961947e-02  2.05688363e+00 -4.90012460e-01]
 [-2.58241077e-01 -1.53409013e-01 -2.70358199e-01 -2.01994942e+00
  -6.78651394e-02  2.06625252e+00 -4.88275527e-01]
 [-2.95732207e-01 -1.02768789e-01 -3.09822572e-01 -2.02264140e+00
  -7.79850005e-02  2.075

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [7]:
del komo
del C

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.0])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [9]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 1.20312, evals: 698, done: 1, feasible: 1, sos: 61.1134, f: 0, ineq: 0, eq: 0.135414 }
size of path: (10, 7)


## Possible State Solution by RRT

In [10]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

37

In [11]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

{ time: 0.015625, evals: 29, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_panda_coll5) [64,51] 	d=-0


In [12]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [13]:
del komo
del rrt
del C

Now, let us give a state which is impossible for robot arm to reach.

##  Part c) Impossible State Solution by KOMO

In [14]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [15]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.328125, evals: 267, done: 1, feasible: 0, sos: 27.9376, f: 0, ineq: 0, eq: 2.85125 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [16]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [17]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: { time: 0.515625, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }
0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0


So we will get an error since there is no feasible solution

In [18]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range

In [41]:
# https://marctoussaint.github.io/robotics-course/tutorials.html -> Tutorials
# https://github.com/MarcToussaint/rai-tutorials/tree/main -> Tutorials
# https://marctoussaint.github.io/robotics-course/rai.html -> Documentation